In [1]:
!pip install python-socketio
!pip install eventlet

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



                                              0.0/57.0 kB ? eta -:--:--
     ---------------------------------------- 57.0/57.0 kB 3.1 MB/s eta 0:00:00
                                              0.0/53.5 kB ? eta -:--:--
     ---------------------------------------- 53.5/53.5 kB 2.9 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable
                                              0.0/226.8 kB ? eta -:--:--
     -------                                  41.0/226.8 kB ? eta -:--:--
     ---------------                         92.2/226.8 kB 1.1 MB/s eta 0:00:01
     ---------------                         92.2/226.8 kB 1.1 MB/s eta 0:00:01
     ---------------                         92.2/226.8 kB 1.1 MB/s eta 0:00:01
     -----------------                    112.6/226.8 kB 504.4 kB/s eta 0:00:01
     --------------------------           163.8/226.8 kB 579.6 kB/s eta 0:00:01
     --------------------------           163.8/226.8 kB 579.6 kB/s et


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
#parsing command line arguments
import argparse
#decoding camera images
import base64
#for frametimestamp saving
from datetime import datetime
#reading and writing files
import os
#high level file operations
import shutil
#matrix math
import numpy as np
#real-time server
import socketio
#concurrent networking 
import eventlet
#web server gateway interface
import eventlet.wsgi
#image manipulation
from PIL import Image
#web framework
from flask import Flask
#input output
from io import BytesIO

#load our saved model
from keras.models import load_model

#helper class
import utils

#initialize our server
sio = socketio.Server()
#our flask (web) app
app = Flask(_name_)
#init our model and image array as empty
model = None
prev_image_array = None

#set min/max speed for our autonomous car
MAX_SPEED = 25
MIN_SPEED = 10

#and a speed limit
speed_limit = MAX_SPEED

#registering event handler for the server
@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        # The current steering angle of the car
        steering_angle = float(data["steering_angle"])
        # The current throttle of the car, how hard to push peddle
        throttle = float(data["throttle"])
        # The current speed of the car
        speed = float(data["speed"])
        # The current image from the center camera of the car
        image = Image.open(BytesIO(base64.b64decode(data["image"])))
        try:
            image = np.asarray(image)       # from PIL image to numpy array
            image = utils.preprocess(image) # apply the preprocessing
            image = np.array([image])       # the model expects 4D array

            # predict the steering angle for the image
            steering_angle = float(model.predict(image, batch_size=1))
            # lower the throttle as the speed increases
            # if the speed is above the current speed limit, we are on a downhill.
            # make sure we slow down first and then go back to the original max speed.
            global speed_limit
            if speed > speed_limit:
                speed_limit = MIN_SPEED  # slow down
            else:
                speed_limit = MAX_SPEED
            throttle = 1.0 - steering_angle**2 - (speed/speed_limit)**2

            print('{} {} {}'.format(steering_angle, throttle, speed))
            send_control(steering_angle, throttle)
        except Exception as e:
            print(e)

        # save frame
        if args.image_folder != '':
            timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
            image_filename = os.path.join(args.image_folder, timestamp)
            image.save('{}.jpg'.format(image_filename))
    else:
        
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle._str_(),
            'throttle': throttle._str_()
        },
        skip_sid=True)


if _name_ == '_main_':
    parser = argparse.ArgumentParser(description='Remote Driving')
    parser.add_argument(
        'model',
        type=str,
        help='Path to model h5 file. Model should be on the same path.'
    )
    parser.add_argument(
        'image_folder',
        type=str,
        nargs='?',
        default='',
        help='Path to image folder. This is where the images from the run will be saved.'
    )
    args = parser.parse_args()

    #load model
    model = load_model(args.model)

    if args.image_folder != '':
        print("Creating image folder at {}".format(args.image_folder))
        if not os.path.exists(args.image_folder):
            os.makedirs(args.image_folder)
        else:
            shutil.rmtree(args.image_folder)
            os.makedirs(args.image_folder)
        print("RECORDING THIS RUN ...")
    else:
        print("NOT RECORDING THIS RUN ...")

    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('',4567)),app)


ModuleNotFoundError: No module named 'tensorflow'

In [6]:
pip install keras


Defaulting to user installation because normal site-packages is not writeable
                                              0.0/1.7 MB ? eta -:--:--
     --                                       0.1/1.7 MB 3.6 MB/s eta 0:00:01
     -----                                    0.2/1.7 MB 4.7 MB/s eta 0:00:01
     -----                                    0.2/1.7 MB 1.9 MB/s eta 0:00:01
     -----                                    0.2/1.7 MB 1.9 MB/s eta 0:00:01
     -----                                    0.2/1.7 MB 1.9 MB/s eta 0:00:01
     -----                                    0.2/1.7 MB 1.9 MB/s eta 0:00:01
     -----                                    0.2/1.7 MB 1.9 MB/s eta 0:00:01
     ------                                   0.3/1.7 MB 780.5 kB/s eta 0:00:02
     ------                                   0.3/1.7 MB 780.5 kB/s eta 0:00:02
     ------                                   0.3/1.7 MB 656.7 kB/s eta 0:00:03
     ------                                   0.3/1.7 MB 656.7 kB


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.13.0-cp310-cp310-win_amd64.whl (1.9 kB)
                                              0.0/276.5 MB ? eta -:--:--
                                              0.1/276.5 MB 2.8 MB/s eta 0:01:38
                                              0.4/276.5 MB 4.6 MB/s eta 0:01:01
                                              0.6/276.5 MB 4.9 MB/s eta 0:00:57
                                              1.1/276.5 MB 5.1 MB/s eta 0:00:54
                                              1.3/276.5 MB 5.2 MB/s eta 0:00:53
                                              1.6/276.5 MB 5.4 MB/s eta 0:00:51
                                              2.0/276.5 MB 5.7 MB/s eta 0:00:48
                                              2.1/276.5 MB 5.4 MB/s eta 0:00:51
                                              2.3/276.5 MB 5.1 MB/s eta 0:00:55
                                              2.4/276.5 MB 5.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
